In [1]:
    # 1- Crie uma consulta que mostre, nesta ordem:
        # Nome, Estados e Status

df = spark.read.format("parquet").load("/home/anthony/pyspark/Arquivos adicionais/Atividades/Clientes.parquet")
dfv = df.withColumnRenamed("Cliente", "Nome")
dfvendas = dfv.withColumnRenamed("Estado", "Estados")

In [37]:
Atv1 = dfvendas.select('Nome', 'Estados', 'Status')
Atv1.show()

+--------------------+-------+--------+
|                Nome|Estados|  Status|
+--------------------+-------+--------+
|Adelina Buenaventura|     RJ|  Silver|
|        Adelino Gago|     RJ|  Silver|
|     Adolfo Patrício|     PE|  Silver|
|    Adriana Guedelha|     RO|Platinum|
|       Adélio Lisboa|     SE|  Silver|
|       Adérito Bahía|     MA|  Silver|
|       Aida Dorneles|     RN|  Silver|
|   Alarico Quinterno|     AC|  Silver|
|    Alberto Cezimbra|     AM|  Silver|
|    Alberto Monsanto|     RN|    Gold|
|       Albino Canela|     AC|  Silver|
|     Alceste Varanda|     RR|  Silver|
|  Alcides Carvalhais|     RO|  Silver|
|        Aldo Martins|     GO|  Silver|
|   Alexandra Tabares|     MG|  Silver|
|      Alfredo Cotrim|     SC|  Silver|
|     Almeno Figueira|     SC|  Silver|
|      Alvito Peralta|     AM|  Silver|
|     Amadeu Martinho|     RN|  Silver|
|      Amélia Estévez|     PE|  Silver|
+--------------------+-------+--------+
only showing top 20 rows



In [52]:
    # 2- Crie uma consulta que mostre apenas os clientes do Status "Platinum" e "Gold"

from pyspark.sql import functions as Func
from pyspark.sql.functions import *

Atv2 = dfvendas.select("*").where((Func.col("Status")=="Platinum") | (Func.col("Status")=="Gold"))
Atv2.show()

+---------+-------------------+-------+------+--------+
|ClienteID|               Nome|Estados|Genero|  Status|
+---------+-------------------+-------+------+--------+
|        4|   Adriana Guedelha|     RO|     F|Platinum|
|       10|   Alberto Monsanto|     RN|     M|    Gold|
|       28|      Anna Carvajal|     RS|     F|    Gold|
|       49|      Bento Quintão|     SP|     M|    Gold|
|       68|      Carminda Dias|     AM|     F|    Gold|
|       83|      Cláudio Jorge|     TO|     M|    Gold|
|      121|    Dionísio Saltão|     PR|     M|    Gold|
|      166|   Firmino Meireles|     AM|     M|    Gold|
|      170|      Flor Vilanova|     CE|     M|Platinum|
|      220|Honorina Villaverde|     PE|     F|    Gold|
|      230|    Ibijara Botelho|     RR|     F|Platinum|
|      237|  Iracema Rodríguez|     BA|     F|    Gold|
|      247|         Joana Ataí|     GO|     F|Platinum|
+---------+-------------------+-------+------+--------+



In [9]:
    # 3- Demonstre quanto cada Status de Clientes representa em vendas.
    
dfvalores = spark.read.format("parquet").load("/home/anthony/pyspark/Arquivos adicionais/Atividades/Vendas.parquet")
dfvalores.show()

+--------+----------+---------+---------+--------+
|VendasID|VendedorID|ClienteID|     Data|   Total|
+--------+----------+---------+---------+--------+
|       1|         1|       91| 1/1/2019|  8053.6|
|       2|         6|      185| 1/1/2020|   150.4|
|       3|         7|       31| 2/1/2020|  6087.0|
|       4|         5|       31| 2/1/2019| 13828.6|
|       5|         5|       31| 3/1/2018|26096.66|
|       6|         5|       31| 4/1/2020| 18402.0|
|       7|         5|       31| 6/1/2019|  7524.2|
|       8|         5|      186| 6/1/2019| 12036.6|
|       9|         7|       91| 6/1/2020| 2804.75|
|      10|         2|      202| 6/1/2020|  8852.0|
|      11|         7|       58| 8/1/2019|16545.25|
|      12|         7|       58| 9/1/2018|11411.88|
|      13|         7|       58|10/1/2019| 15829.7|
|      14|         3|      249|12/1/2020| 6154.36|
|      15|         4|      249|12/1/2018| 3255.08|
|      16|         7|      192|13/1/2020| 2901.25|
|      17|         2|       79|

In [11]:
inner = (dfvendas.join(dfvalores, on=["ClienteID"], how='inner'))
inner.show()

+---------+-------------------+-------+------+------+--------+----------+---------+--------+
|ClienteID|               Nome|Estados|Genero|Status|VendasID|VendedorID|     Data|   Total|
+---------+-------------------+-------+------+------+--------+----------+---------+--------+
|       91|     Cosme Zambujal|     BA|     M|Silver|       1|         1| 1/1/2019|  8053.6|
|      185|  Gertrudes Hidalgo|     PA|     F|Silver|       2|         6| 1/1/2020|   150.4|
|       31|   Antão Corte-Real|     ES|     M|Silver|       3|         7| 2/1/2020|  6087.0|
|       31|   Antão Corte-Real|     ES|     M|Silver|       4|         5| 2/1/2019| 13828.6|
|       31|   Antão Corte-Real|     ES|     M|Silver|       5|         5| 3/1/2018|26096.66|
|       31|   Antão Corte-Real|     ES|     M|Silver|       6|         5| 4/1/2020| 18402.0|
|       31|   Antão Corte-Real|     ES|     M|Silver|       7|         5| 6/1/2019|  7524.2|
|      186|  Gertrudes Infante|     RS|     F|Silver|       8|        

In [39]:
Atv3 = inner.groupBy("Status").agg(sum("Total")).orderBy(Func.col("sum(Total)").desc())
Atv3.show()

+--------+------------------+
|  Status|        sum(Total)|
+--------+------------------+
|  Silver|        3014291.36|
|    Gold|27286.690000000002|
|Platinum|          12584.68|
+--------+------------------+



In [32]:
    ## Repostas do professor
            # 1- Crie uma consulta que mostre, nesta ordem:
                # Nome, Estados e Status
clientes = spark.read.load("/home/anthony/pyspark/Arquivos adicionais/Atividades/Clientes.parquet")
clientes.show(2)

+---------+--------------------+------+------+------+
|ClienteID|             Cliente|Estado|Genero|Status|
+---------+--------------------+------+------+------+
|        1|Adelina Buenaventura|    RJ|     M|Silver|
|        2|        Adelino Gago|    RJ|     M|Silver|
+---------+--------------------+------+------+------+
only showing top 2 rows



In [33]:
vendas = spark.read.load("/home/anthony/pyspark/Arquivos adicionais/Atividades/Vendas.parquet")
vendas.show(2)

+--------+----------+---------+--------+------+
|VendasID|VendedorID|ClienteID|    Data| Total|
+--------+----------+---------+--------+------+
|       1|         1|       91|1/1/2019|8053.6|
|       2|         6|      185|1/1/2020| 150.4|
+--------+----------+---------+--------+------+
only showing top 2 rows



In [36]:
Atividade1 = clientes.select("Cliente", "Estado", "Status")
Atividade1.show()

+--------------------+------+--------+
|             Cliente|Estado|  Status|
+--------------------+------+--------+
|Adelina Buenaventura|    RJ|  Silver|
|        Adelino Gago|    RJ|  Silver|
|     Adolfo Patrício|    PE|  Silver|
|    Adriana Guedelha|    RO|Platinum|
|       Adélio Lisboa|    SE|  Silver|
|       Adérito Bahía|    MA|  Silver|
|       Aida Dorneles|    RN|  Silver|
|   Alarico Quinterno|    AC|  Silver|
|    Alberto Cezimbra|    AM|  Silver|
|    Alberto Monsanto|    RN|    Gold|
|       Albino Canela|    AC|  Silver|
|     Alceste Varanda|    RR|  Silver|
|  Alcides Carvalhais|    RO|  Silver|
|        Aldo Martins|    GO|  Silver|
|   Alexandra Tabares|    MG|  Silver|
|      Alfredo Cotrim|    SC|  Silver|
|     Almeno Figueira|    SC|  Silver|
|      Alvito Peralta|    AM|  Silver|
|     Amadeu Martinho|    RN|  Silver|
|      Amélia Estévez|    PE|  Silver|
+--------------------+------+--------+
only showing top 20 rows



In [42]:
from pyspark.sql import functions as Func
Atividade2 = clientes.select("*").where((Func.col("Status")=="Gold") | (Func.col("Status")=="Platinum"))
Atividade2.show()

+---------+-------------------+------+------+--------+
|ClienteID|            Cliente|Estado|Genero|  Status|
+---------+-------------------+------+------+--------+
|        4|   Adriana Guedelha|    RO|     F|Platinum|
|       10|   Alberto Monsanto|    RN|     M|    Gold|
|       28|      Anna Carvajal|    RS|     F|    Gold|
|       49|      Bento Quintão|    SP|     M|    Gold|
|       68|      Carminda Dias|    AM|     F|    Gold|
|       83|      Cláudio Jorge|    TO|     M|    Gold|
|      121|    Dionísio Saltão|    PR|     M|    Gold|
|      166|   Firmino Meireles|    AM|     M|    Gold|
|      170|      Flor Vilanova|    CE|     M|Platinum|
|      220|Honorina Villaverde|    PE|     F|    Gold|
|      230|    Ibijara Botelho|    RR|     F|Platinum|
|      237|  Iracema Rodríguez|    BA|     F|    Gold|
|      247|         Joana Ataí|    GO|     F|Platinum|
+---------+-------------------+------+------+--------+



In [46]:
Atividade3 = vendas.join(clientes, vendas.ClienteID == clientes.ClienteID).groupBy(clientes.Status).agg(sum("Total")).orderBy(Func.col("sum(Total)").desc())
Atividade3.show()

+--------+------------------+
|  Status|        sum(Total)|
+--------+------------------+
|  Silver|        3014291.36|
|    Gold|27286.690000000002|
|Platinum|          12584.68|
+--------+------------------+



In [47]:
Atv1.show()

+--------------------+-------+--------+
|                Nome|Estados|  Status|
+--------------------+-------+--------+
|Adelina Buenaventura|     RJ|  Silver|
|        Adelino Gago|     RJ|  Silver|
|     Adolfo Patrício|     PE|  Silver|
|    Adriana Guedelha|     RO|Platinum|
|       Adélio Lisboa|     SE|  Silver|
|       Adérito Bahía|     MA|  Silver|
|       Aida Dorneles|     RN|  Silver|
|   Alarico Quinterno|     AC|  Silver|
|    Alberto Cezimbra|     AM|  Silver|
|    Alberto Monsanto|     RN|    Gold|
|       Albino Canela|     AC|  Silver|
|     Alceste Varanda|     RR|  Silver|
|  Alcides Carvalhais|     RO|  Silver|
|        Aldo Martins|     GO|  Silver|
|   Alexandra Tabares|     MG|  Silver|
|      Alfredo Cotrim|     SC|  Silver|
|     Almeno Figueira|     SC|  Silver|
|      Alvito Peralta|     AM|  Silver|
|     Amadeu Martinho|     RN|  Silver|
|      Amélia Estévez|     PE|  Silver|
+--------------------+-------+--------+
only showing top 20 rows



In [48]:
Atividade1.show()

+--------------------+------+--------+
|             Cliente|Estado|  Status|
+--------------------+------+--------+
|Adelina Buenaventura|    RJ|  Silver|
|        Adelino Gago|    RJ|  Silver|
|     Adolfo Patrício|    PE|  Silver|
|    Adriana Guedelha|    RO|Platinum|
|       Adélio Lisboa|    SE|  Silver|
|       Adérito Bahía|    MA|  Silver|
|       Aida Dorneles|    RN|  Silver|
|   Alarico Quinterno|    AC|  Silver|
|    Alberto Cezimbra|    AM|  Silver|
|    Alberto Monsanto|    RN|    Gold|
|       Albino Canela|    AC|  Silver|
|     Alceste Varanda|    RR|  Silver|
|  Alcides Carvalhais|    RO|  Silver|
|        Aldo Martins|    GO|  Silver|
|   Alexandra Tabares|    MG|  Silver|
|      Alfredo Cotrim|    SC|  Silver|
|     Almeno Figueira|    SC|  Silver|
|      Alvito Peralta|    AM|  Silver|
|     Amadeu Martinho|    RN|  Silver|
|      Amélia Estévez|    PE|  Silver|
+--------------------+------+--------+
only showing top 20 rows



In [53]:
Atv2.show()

+---------+-------------------+-------+------+--------+
|ClienteID|               Nome|Estados|Genero|  Status|
+---------+-------------------+-------+------+--------+
|        4|   Adriana Guedelha|     RO|     F|Platinum|
|       10|   Alberto Monsanto|     RN|     M|    Gold|
|       28|      Anna Carvajal|     RS|     F|    Gold|
|       49|      Bento Quintão|     SP|     M|    Gold|
|       68|      Carminda Dias|     AM|     F|    Gold|
|       83|      Cláudio Jorge|     TO|     M|    Gold|
|      121|    Dionísio Saltão|     PR|     M|    Gold|
|      166|   Firmino Meireles|     AM|     M|    Gold|
|      170|      Flor Vilanova|     CE|     M|Platinum|
|      220|Honorina Villaverde|     PE|     F|    Gold|
|      230|    Ibijara Botelho|     RR|     F|Platinum|
|      237|  Iracema Rodríguez|     BA|     F|    Gold|
|      247|         Joana Ataí|     GO|     F|Platinum|
+---------+-------------------+-------+------+--------+



In [50]:
Atividade2.show()

+---------+-------------------+------+------+--------+
|ClienteID|            Cliente|Estado|Genero|  Status|
+---------+-------------------+------+------+--------+
|        4|   Adriana Guedelha|    RO|     F|Platinum|
|       10|   Alberto Monsanto|    RN|     M|    Gold|
|       28|      Anna Carvajal|    RS|     F|    Gold|
|       49|      Bento Quintão|    SP|     M|    Gold|
|       68|      Carminda Dias|    AM|     F|    Gold|
|       83|      Cláudio Jorge|    TO|     M|    Gold|
|      121|    Dionísio Saltão|    PR|     M|    Gold|
|      166|   Firmino Meireles|    AM|     M|    Gold|
|      170|      Flor Vilanova|    CE|     M|Platinum|
|      220|Honorina Villaverde|    PE|     F|    Gold|
|      230|    Ibijara Botelho|    RR|     F|Platinum|
|      237|  Iracema Rodríguez|    BA|     F|    Gold|
|      247|         Joana Ataí|    GO|     F|Platinum|
+---------+-------------------+------+------+--------+



In [54]:
Atv3.show()

+--------+------------------+
|  Status|        sum(Total)|
+--------+------------------+
|  Silver|        3014291.36|
|    Gold|27286.690000000002|
|Platinum|          12584.68|
+--------+------------------+



In [55]:
Atividade3.show()

+--------+------------------+
|  Status|        sum(Total)|
+--------+------------------+
|  Silver|        3014291.36|
|    Gold|27286.690000000002|
|Platinum|          12584.68|
+--------+------------------+

